# Clasificador de sesiones

La idea es desarrollar una pequeña prueba de una función que clasifique una sesión de juegos en una dificultad determinada, de las siguientes:

1. Muy fácil.
2. Fácil.
3. Intermedia.
4. Difícil.
5. Muy difícil.

Como objetivo adicional, el clasificador debe poder funcionar a partir de las sesiones de juego. Esto es para que cada sesión de juego tenga una única dificultad. De otro modo, si usáramos los resultados, valores de resultados de la misma sesión podrían terminar en dificultades diferentes.

## Parámetros

No se considera el tiempo para la clasificación. Se usa como base para la función de clasificación las dificultades definidas para los presets. La idea siendo que la función debería ajustar de forma más o menos acorde a lo que el equipo ya definió previamente.

Se adjuntan las tablas de dificultad para cada juego a continuación.

### Encuentra al repetido

| Level | MaxStimuli | VariableSize | Distractors | FigureSet |  Difficulty |
|:-----:|:----------:|:------------:|:-----------:|:---------:|:-----------:|
|   5   |      5     |     false    |    false    |   Frutas  |  Muy fácil  |
|   10  |      9     |     false    |    false    |   Frutas  |    Fácil    |
|   15  |     12     |     false    |     true    |   Frutas  |  Intermedia |
|   20  |     17     |     true     |    false    |   Flores  |   Difícil   |
|   20  |     20     |     true     |     true    |   Flores  | Muy difícil |

### Encuentra al repetido

| Level | VariableSize | FigureSet |  Difficulty |
|:-----:|:------------:|:---------:|:-----------:|
|   5   |     false    |   Frutas  |  Muy fácil  |
|   7   |     false    |   Frutas  |    Fácil    |
|   10  |     true     |   Frutas  |  Intermedia |
|   14  |     false    |   Flores  |   Difícil   |
|   17  |     true     |   Flores  | Muy difícil |

### Memorilla

| Level | MaxStimuli | Rows | Columns |  Difficulty |
|:-----:|:----------:|:----:|:-------:|:-----------:|
|   3   |      3     |   4  |    4    |  Muy fácil  |
|   5   |      5     |   5  |    5    |    Fácil    |
|   7   |      7     |   6  |    5    |  Intermedia |
|   8   |      9     |   6  |    6    |   Difícil   |
|   10  |     10     |   8  |    6    | Muy difícil |

In [61]:
import numpy as np
import pandas as pd
%run ./classifier.ipynb import EARClassifier, EANClassifier, MClassifier, FigureSet

# Introducción

## Objetivos

El clasificador tiene los siguientes objetivos:

- Asignar una dificultad a cada sesión de juego, teniendo un total de 5 dificultades, usando pura y exclusivamente sus parámetros.
- Emparejar sesiones de juego con dificultades similares en pos de poder calcular el MGP usando valores con un scope más amplio.
- Utilizar un cálculo generalizable y que permite buena escalabilidad para aplicar en diferentes juegos.

Como objetivo opcional, el equipo de trabajo reconoce que cada parámetro afecta en diferentes medidas a la dificultad, y por tanto, sería útil, tener una forma de controlar el nivel de influencia con el cual cada parámetro afecta el valor de dificultad.

## Aproximación

Considerando todos estos aspectos, la solución más apropiada en mi opinión (sin entrar en el campo de la Inteligencia Artificial) es asignar una función individual de clasificación a cada parámetro, y aplicar una media ponderada. Esto nos permite:

- Escalar y mantener la solución a cualquier juego, dado que no depende de los parámetros involucrados.
- Regular el nivel de influencia de cada parámetro a través de la ponderación.

Tomaremos entonces como base las tablas de dificultades utilizadas anteriormente, y las usaremos para encontrar una función de dificultad para cada parámetro, usando diferentes metodologías de aproximación de funciones.

## Diseño

Se creará una clase `GClassifier` para cada juego `Game`, la cual tiene los parámetros de la sesión, con sus valores, y los siguientes métodos:

- Por cada `param` se crea un método `paramDifficulty` que está definido por la función dificultad.
- Por cada `param` se crea un atributo `paramPonderation` que está definido por la ponderación del `param` y es estático.
- Tiene una función `classify` que clasifica la sesión en la dificultad correspondiente usando una media ponderada de cada `param`.

Todos los clasificadores están definidos en el archivo `classifier.ipynb`.

Además de esto, consideramos útil que el valor de dificultad sea un número entre 1 y 5, donde 1 es la dificultad más fácil y 5 la más difícil. La idea es que este valor puede ser fraccionario, y se redondea para determinar la dificultad final.

In [73]:
print("Documentación EARClassifier:")
print(EARClassifier.__doc__)
print("Documentación levelDifficulty:")
print(EARClassifier.levelDifficulty.__doc__)
print("Documentación classify:")
print(EARClassifier.classify.__doc__)

Documentación EARClassifier:

    Clasificador de sesiones de Encuentra al Repetido.
    Se usa para asignar una dificultad a las sesiones de 
    Encuentra al Repetido.

    Attributes:
        level:              Cantidad de niveles de la sesión.
        maxStimuli:         Cantidad de máxima de estímulos que puede tener la sesión.
        variableSize:       Están activados los tamaños variables para los estímulos.
        distractors:        Están activados los distractores.
        figureSet:          Set de figuras.
    
Documentación levelDifficulty:

        Calcula la dificultad basándose en la cantidad de niveles de la sesión.

        Returns:
        float: Dificultad del nivel.
        
Documentación classify:

        Clasifica la sesión en una dificultad del enum `Difficulty`.

        Returns:
        Difficulty: Dificultad de la sesión.
        


# Encuentra al Repetido

## Nivel

Empezamos con el juego Encuentra al Repetido. Lo primero que hice, fue usar el parámetro de niveles (`Level`), y usando la tabla plotearlo en GeoGebra. En esta primera instancia, las funciones fueron creadas a mano, usando la herramienta recta y mis conocimientos de análisis matemático 1 para crear una parábola.

El resultado es el siguiente:

![Ajuste de Curva de EAR para cantidad de niveles](./img/level-adjustment-1.png)

En esta instancia, me pareció que la función parabólica era lo suficientemente similar a lo que estaba buscando, por lo que es la que se usó finalmente para el clasificador.

Una cosa que cabe aclarar es que algo que es muy útil para estas funciones es que los valores límite del parámetro, sean iguales al mínimo (1) y máximo (5) de la función de dificultad. Esto, nos permite garantizar que los valores de dificultad van a estar siempre entre 1 y 5, lo que facilita el cálculo del promedio.

## Cantidad de estímulos

Para el caso de la cantidad de estímulos, me costó hacer una aproximación adivinada que se ajuste a la curva de forma correcta. Estos fueron mis mejores intentos:

![imagen](./img/stimuli-adjustment-1.png)

No conforme con esto, decidí realizar una breve investigación sobre aproximación de funciones usando métodos polinómicos, descubriendo que están incorporados en GeoGebra:

![ajuste polinomico](./img/ajuste-polinomico.png)

En este punto, no estaba muy seguro de qué curva se ajustaba mejor a la dificultad definida, por lo que decidí que lo mejor era calcular el error usando este mismo notebook:

In [63]:
## Error cantidad estimulos EAR
def f(x):
    '''
    Función que aproxima la dificultad de la cantidad de estímulos
    usando una recta.

    Parameters:
    x (float): Cantidad de estímulos.
    
    Returns:
    float: Dificultad de la cantidad de estímulos.
    '''
    return 0.25 * x - 0.25

def g(x):
    '''
    Función que aproxima la dificultad de la cantidad de estímulos
    usando una función cuadrática.

    Parameters:
    x (float): Cantidad de estímulos.
    
    Returns:
    float: Dificultad de la cantidad de estímulos.
    '''
    return 0.01 * x ** 2 + 1

def h(x):
    '''
    Función que aproxima la dificultad de la cantidad de estímulos
    usando un polinomio de 4to orden.

    Parameters:
    x (float): Cantidad de estímulos.
    
    Returns:
    float: Dificultad de la cantidad de estímulos.
    '''
    return 0.00001 * x ** 4 + 0.008 * x **2 + 1

# Valores originales de dificultad y cantidad de estímulos
x = [0, 5, 9, 12, 17, 20]
y = [1, 1, 2, 3, 4, 5]

def calculateError(f):
    '''
    Calcula el error de una función.

    Parameters:
    f (function): Función de aproximación.
    
    Returns:
    float: Error de la función.
    '''
    error = 0

    for i in range(len(x)):
        error += f(x[i]) - y[i]
        
    return error / len(x)

print("Error recta: ", calculateError(f))
print("Error cuadrática: ", calculateError(g))
print("Error 4to orden: ", calculateError(h))

Error recta:  -0.2916666666666667
Error cuadrática:  -0.10166666666666664
Error 4to orden:  0.037738333333333395


Por esta razón, terminé usando la función de 4to órden, dado que es la que tiene menor error.

## Valores binarios

Los otros 3 parámetros solamente pueden tomar dos valores. Dos son parámetros booleanos, y el tercero es el conjunto de figuras, el cual en este momento solo hay dos implementados.

Por esto, decidí usar una solución muy sencilla, y recurrir a un `if` y determinar el valor más difícil del parámetro como 5, y el más bajo como 1.

## Ajustes

Una vez hecho esto, otra cosa que tuve que acomodar era limintar el rango de algunas funciones, que subían por encima de 5.

Lo siguiente fue probar el clasificador, y quedé sorprendido de que funcionaba bastante bien sin usar ninguna ponderación:

In [64]:
## Valores originales de la tabla con ponderación 1

c1 = EARClassifier(5, 5, False, False, FigureSet.Frutas)
c1.levelPonderation = 1
c1.maxStimuliPonderation = 1
c1.variableSizePonderation = 1
c1.distractorsPonderation = 1
c1.figureSetPonderation = 1
c2 = EARClassifier(10, 9, False, False, FigureSet.Frutas)
c2.levelPonderation = 1
c2.maxStimuliPonderation = 1
c2.variableSizePonderation = 1
c2.distractorsPonderation = 1
c2.figureSetPonderation = 1
c3 = EARClassifier(15, 12, False, True, FigureSet.Frutas)
c3.levelPonderation = 1
c3.maxStimuliPonderation = 1
c3.variableSizePonderation = 1
c3.distractorsPonderation = 1
c3.figureSetPonderation = 1
c4 = EARClassifier(20, 17, True, False, FigureSet.Flores)
c4.levelPonderation = 1
c4.maxStimuliPonderation = 1
c4.variableSizePonderation = 1
c4.distractorsPonderation = 1
c4.figureSetPonderation = 1
c5 = EARClassifier(20, 20, True, True, FigureSet.Flores)
c5.levelPonderation = 1
c5.maxStimuliPonderation = 1
c5.variableSizePonderation = 1
c5.distractorsPonderation = 1
c5.figureSetPonderation = 1

# Este valor fue agregado después para experimentar.
# Se cambia solamente si están activados los distractores, partiendo de la
# suposición de que la dificultad no debería cambiar.

c6 = EARClassifier(20, 20, True, False, FigureSet.Flores)
c6.levelPonderation = 1
c6.maxStimuliPonderation = 1
c6.variableSizePonderation = 1
c6.distractorsPonderation = 1
c6.figureSetPonderation = 1

print("Dificultad muy fácil: ", c1.classifyRaw(), c1.classify().name)
print("Dificultad fácil: ", c2.classifyRaw(), c2.classify().name)
print("Dificultad intermedia: ", c3.classifyRaw(), c3.classify().name)
print("Dificultad difícil: ", c4.classifyRaw(), c4.classify().name)
print("Dificultad muy difícil: ", c5.classifyRaw(), c5.classify().name)
print("Este no debería cambiar: ", c6.classifyRaw(), c6.classify().name)

Dificultad muy fácil:  1.09125 MuyFacil
Dificultad fácil:  1.342722 MuyFacil
Dificultad intermedia:  2.521872 Intermedia
Dificultad difícil:  4.029442 Dificil
Dificultad muy difícil:  5.0 MuyDificil
Este no debería cambiar:  4.2 Dificil


Vemos que el cambio de dificultad entre uno y otro es tal, que el solo hecho de cambiar si hay distractores o no, nos cambia la dificultad del ejercicio. En este punto, el clasificador se analizó junto con el equipo, en pos de encontrar la mejor ponderación. Luego de varios experimentos, se llegó a la ponderación final:

| Parámetro    | Ponderación |
|--------------|------------:|
| Level        |        1,75 |
| MaxStimuli   |        2,25 |
| VariableSize |        0,25 |
| Distractors  |        0,25 |
| FigureSet    |         0,5 |

Como se ve en el siguiente resultado, los valores son mucho más adecuados, dado que la brecha entre distractores activados y desactivados no es tan grande. De la misma forma, la diferencia entre usar flores como estímulos y frutas es significativa, lo que se corresponde con la dificultad del ejercicio.

In [65]:
c1 = EARClassifier(5, 5, False, False, FigureSet.Frutas)
c2 = EARClassifier(10, 9, False, False, FigureSet.Frutas)
c3 = EARClassifier(15, 12, False, True, FigureSet.Frutas)
c4 = EARClassifier(20, 17, True, False, FigureSet.Flores)
c5 = EARClassifier(20, 20, True, True, FigureSet.Flores)
c6 = EARClassifier(20, 20, True, False, FigureSet.Flores)
c7 = EARClassifier(20, 20, True, True, FigureSet.Frutas)

sessions = [
    [5, 5, False, False, FigureSet.Frutas, c1.classifyRaw(), c1.classify().name],
    [10, 9, False, False, FigureSet.Frutas, c2.classifyRaw(), c2.classify().name],
    [15, 12, False, True, FigureSet.Frutas, c3.classifyRaw(), c3.classify().name],
    [20, 17, True, False, FigureSet.Flores, c4.classifyRaw(), c4.classify().name],
    [20, 20, True, True, FigureSet.Flores, c5.classifyRaw(), c5.classify().name],
    [20, 20, True, False, FigureSet.Flores, c6.classifyRaw(), c6.classify().name],
    [20, 20, True, True, FigureSet.Frutas, c7.classifyRaw(), c7.classify().name]
]

pd.DataFrame(data=sessions, columns=['Level','MaxStimuli', 'VariableSize', 'Distractors', 'FigureSet', 'DifficultyValue', 'Difficulty']).head(10)

,Level,MaxStimuli,VariableSize,Distractors,FigureSet,DifficultyValue,Difficulty
0,5,5,False,False,FigureSet.Frutas,1.180313,MuyFacil
1,10,9,False,False,FigureSet.Frutas,1.671124,Facil
2,15,12,False,True,FigureSet.Frutas,2.599212,Intermedia
3,20,17,True,False,FigureSet.Flores,4.416244,Dificil
4,20,20,True,True,FigureSet.Flores,5.000000,MuyDificil
5,20,20,True,False,FigureSet.Flores,4.800000,MuyDificil
6,20,20,True,True,FigureSet.Frutas,4.600000,MuyDificil


# Encuentra al Nuevo

El resto del procedimiento fue bastante similar. Usando GeoGebra encontraba la función más adecuada para los valores numéricos, calculaba el error, elegía el más adecuado y ajustaba la ponderación.

En el caso particular de Encuentra al Nuevo, esto fue particularmente sencillo, porque todos los parámetros son los mismos. Cambia ligeramente la función de ajuste:

![ajuste de nivel](./img/level-adjustment-2.png)

In [66]:
def f(x):
    '''
    Función que aproxima la dificultad de la cantidad de niveles
    usando una recta.

    Parameters:
    x (float): Cantidad de estímulos.
    
    Returns:
    float: Dificultad de la cantidad de estímulos.
    '''
    return (1/3) * x - (2/3)

def g(x):
    '''
    Función que aproxima la dificultad de la cantidad de niveles
    usando una recta.

    Parameters:
    x (float): Cantidad de estímulos.
    
    Returns:
    float: Dificultad de la cantidad de estímulos.
    '''
    return 0.375 * x - 0.875


x = [5, 7, 10, 13, 17]
y = [1, 2, 3, 4, 5]


def calculateError(f):
    '''
    Calcula el error de una función.

    Parameters:
    f (function): Función de aproximación.
    
    Returns:
    float: Error de la función.
    '''
    error = 0

    for i in range(len(x)):
        error += f(x[i]) - y[i]
        
    return error / len(x)

print("Error recta 1: ", calculateError(f))
print("Error recta 2: ", calculateError(g))

Error recta 1:  -0.20000000000000026
Error recta 2:  0.025


El único cambio que hubo que hacer en la ponderación fue para que la suma de las ponderaciones sea 3.

In [67]:
c1 = EANClassifier(5, False, FigureSet.Frutas)
c2 = EANClassifier(7, False, FigureSet.Frutas)
c3 = EANClassifier(10, True, FigureSet.Frutas)
c4 = EANClassifier(14, False, FigureSet.Flores)
c5 = EANClassifier(17, True, FigureSet.Flores)
c6 = EANClassifier(17, False, FigureSet.Frutas)

sessions = [
    [5, False, FigureSet.Frutas, c1.classifyRaw(), c1.classify().name],
    [7, False, FigureSet.Frutas, c2.classifyRaw(), c2.classify().name],
    [10, True, FigureSet.Frutas, c3.classifyRaw(), c3.classify().name],
    [14, False, FigureSet.Flores, c4.classifyRaw(), c4.classify().name],
    [17, True, FigureSet.Flores, c5.classifyRaw(), c5.classify().name],
    [17, False, FigureSet.Frutas, c6.classifyRaw(), c6.classify().name]
]

pd.DataFrame(data=sessions, columns=['Level', 'VariableSize', 'FigureSet', 'DifficultyValue', 'Difficulty']).head(10)

,Level,VariableSize,FigureSet,DifficultyValue,Difficulty
0,5,False,FigureSet.Frutas,1.000000,MuyFacil
1,7,False,FigureSet.Frutas,1.562500,Facil
2,10,True,FigureSet.Frutas,2.739583,Intermedia
3,14,False,FigureSet.Flores,4.197917,Dificil
4,17,True,FigureSet.Flores,5.000000,MuyDificil
5,17,False,FigureSet.Frutas,4.000000,Dificil


# Memorilla

En el caso de la Memorilla, se repite el proceso, con la diferencia de que hubo que ajustar 4 curvas.

## Nivel

En el caso del nivel, decidí usar una recta:

![ajuste](./img/ajuste-polinomico-nivel-memorilla.png)

Como no estaba seguro de qué puntos iban a dar el mejor ajuste, probé todas las combinaciones y elegí la de menor error:

In [68]:
# Ajuste memorilla nivel
a = (3, 1)
b = (5, 2)
c = (7, 3)
d = (8, 4)
e = (10, 5)

def f(x1, x2):
    '''
    Crea una función recta a partir de dos puntos.

    Parameters:
    x1 (tupla): Punto x1.
    x1 (tupla): Punto x2.
    
    Returns:
    function: Función recta.
    '''
    return lambda x : (x2[1]-x1[1])/(x2[0]-x1[0]) * (x - x1[0]) + x1[1]

x = [a[0], b[0], c[0], d[0], e[0]]
y = [a[1], b[1], c[1], d[1], e[1]]

def calculateError(f):
    '''
    Calcula el error de una función.

    Parameters:
    f (function): Función de aproximación.
    
    Returns:
    float: Error de la función.
    '''
    error = 0

    for i in range(len(x)):
        error += f(x[i]) - y[i]
        
    return error / len(x)

points = [a, b, c, d, e]

def calculateAllErrors():
    '''
    Calcula el error de todas las rectas y lo imprime.
    '''
    for i in range(len(points)):
        for j in range(len(points[:i])):
            print("Error de la recta usando los puntos ", points[i], " y ", points[j], ": ", calculateError(f(points[i], points[j])))

calculateAllErrors()

Error de la recta usando los puntos  (5, 2)  y  (3, 1) :  -0.2
Error de la recta usando los puntos  (7, 3)  y  (3, 1) :  -0.2
Error de la recta usando los puntos  (7, 3)  y  (5, 2) :  -0.2
Error de la recta usando los puntos  (8, 4)  y  (3, 1) :  0.16000000000000006
Error de la recta usando los puntos  (8, 4)  y  (5, 2) :  0.0666666666666667
Error de la recta usando los puntos  (8, 4)  y  (7, 3) :  -0.4
Error de la recta usando los puntos  (10, 5)  y  (3, 1) :  0.057142857142857204
Error de la recta usando los puntos  (10, 5)  y  (5, 2) :  -0.040000000000000036
Error de la recta usando los puntos  (10, 5)  y  (7, 3) :  -0.26666666666666644
Error de la recta usando los puntos  (10, 5)  y  (8, 4) :  0.3


## Cantidad de estímulos

Ocurrió algo similar en la cantidad de estímulos:

![Ajuste recta cantidad de estímulos](./img/ajuste-polinomico-estimulosmemorilla.png)

In [69]:
# Ajuste memorilla estímulos
a = (3, 1)
b = (5, 2)
c = (7, 3)
d = (9, 4)
e = (10, 5)

def f(x1, x2):
    '''
    Crea una función recta a partir de dos puntos.

    Parameters:
    x1 (tupla): Punto x1.
    x1 (tupla): Punto x2.
    
    Returns:
    function: Función recta.
    
    '''
    return lambda x : (x2[1]-x1[1])/(x2[0]-x1[0]) * (x - x1[0]) + x1[1]

x = [a[0], b[0], c[0], d[0], e[0]]
y = [a[1], b[1], c[1], d[1], e[1]]

def calculateError(f):
    '''
    Calcula el error de una función.

    Parameters:
    f (function): Función de aproximación.
    
    Returns:
    float: Error de la función.
    '''
    error = 0

    for i in range(len(x)):
        error += f(x[i]) - y[i]
        
    return error / len(x)

points = [a, b, c, d, e]

def calculateAllErrors():
    '''
    Calcula el error de todas las rectas y lo imprime.
    '''
    for i in range(len(points)):
        for j in range(len(points[:i])):
            print("Error de la recta usando los puntos ", points[i], " y ", points[j], ": ", calculateError(f(points[i], points[j])))

calculateAllErrors()

Error de la recta usando los puntos  (5, 2)  y  (3, 1) :  -0.1
Error de la recta usando los puntos  (7, 3)  y  (3, 1) :  -0.1
Error de la recta usando los puntos  (7, 3)  y  (5, 2) :  -0.1
Error de la recta usando los puntos  (9, 4)  y  (3, 1) :  -0.1
Error de la recta usando los puntos  (9, 4)  y  (5, 2) :  -0.1
Error de la recta usando los puntos  (9, 4)  y  (7, 3) :  -0.1
Error de la recta usando los puntos  (10, 5)  y  (3, 1) :  0.17142857142857154
Error de la recta usando los puntos  (10, 5)  y  (5, 2) :  0.08000000000000007
Error de la recta usando los puntos  (10, 5)  y  (7, 3) :  -0.13333333333333322
Error de la recta usando los puntos  (10, 5)  y  (9, 4) :  -1.2


## Filas

Para las filas, usé un ajuste polinómico:

![Ajuste polinómico de filas](./img/ajuste-polinomico-filas-memorilla.png)

Calculé el error para cada curva, y elegí la del menor error.

## Columnas

Lo mismo para las columnas:

![Ajuste polinómico de columnas](./img/ajuste-polinomico-columnas-memorilla.png)

En este caso, no hice cálculo de error, porque no se pueden calcular polinomios de 3er grado o superior.

Usando estas funciones de dificultad, y ajustando un poco las ponderaciones, llegamos al resultado final:

In [70]:
c1 = MClassifier(3, 3, 4, 4)
c2 = MClassifier(5, 5, 5, 5)
c3 = MClassifier(7, 7, 6, 5)
c4 = MClassifier(8, 9, 6, 6)
c5 = MClassifier(10, 10, 8, 6)
c6 = MClassifier(10, 9, 8, 6)

sessions = [
    [3, 3, 4, 4, c1.classifyRaw(), c1.classify().name],
    [5, 5, 5, 5, c2.classifyRaw(), c2.classify().name],
    [7, 7, 6, 5, c3.classifyRaw(), c3.classify().name],
    [8, 9, 6, 6, c4.classifyRaw(), c4.classify().name],
    [10, 10, 8, 6, c5.classifyRaw(), c5.classify().name],
    [10, 9, 8, 6, c6.classifyRaw(), c6.classify().name]
]

pd.DataFrame(data=sessions, columns=['Level', 'VariableSize', 'Rows', 'Columns', 'DifficultyValue', 'Difficulty']).head(10)

,Level,VariableSize,Rows,Columns,DifficultyValue,Difficulty
0,3,3,4,4,0.8500,MuyFacil
1,5,5,5,5,2.0625,Facil
2,7,7,6,5,3.1500,Intermedia
3,8,9,6,6,4.0750,Dificil
4,10,10,8,6,4.9375,MuyDificil
5,10,9,8,6,4.7125,MuyDificil


Se puede encontrar la lista completa de sesiones posibles y clasificaciones en el archivo [Todas las sesiones clasificadas](./all-sessions-classified.ipynb), así como la documentación e implementación de los clasificadores en [Clasificadores](./classifier.ipynb).